## Demo: Run Multiple Intent Experiments

In [1]:
%%capture
from leaderboard.nlu.base import base_experiment, intent_experiment
import copy

### Chose Configurations

In [2]:
base_config = intent_experiment.IntentExperimentConfiguration(
    exclusion_percentage=0.,
    drop_intents_with_less_than=2,
    data=base_experiment.DataConfig(
        name="examples-rules-nlu",
        data_path="../../../examples/rules/data/nlu.yml",
        domain_path="../../../examples/rules/domain.yml",
    ),
    model=base_experiment.ModelConfig(),
    clear_rasa_cache=True,
)

In [3]:
variations = {
    "interesting-config-name1" : "./config2.yml" ,
    "another-very-interesting-config-because-reasons" : "./config1.yml" ,
}

In [4]:
configs = []
for name, config_path in variations.items():
    config = copy.deepcopy(base_config)
    config.model.name = name
    config.model.config_path = config_path
    configs.append(config)

In [5]:
configs

[IntentExperimentConfiguration(description='Basic Intent Experiment', model=ModelConfig(name='interesting-config-name1', config_path='./config2.yml'), data=DataConfig(name='examples-rules-nlu', domain_path='../../../examples/rules/domain.yml', data_path='../../../examples/rules/data/nlu.yml'), clear_rasa_cache=True, drop_intents_with_less_than=2, exclusion_percentage=0.0),
 IntentExperimentConfiguration(description='Basic Intent Experiment', model=ModelConfig(name='another-very-interesting-config-because-reasons', config_path='./config1.yml'), data=DataConfig(name='examples-rules-nlu', domain_path='../../../examples/rules/domain.yml', data_path='../../../examples/rules/data/nlu.yml'), clear_rasa_cache=True, drop_intents_with_less_than=2, exclusion_percentage=0.0)]

### Run!

In [6]:
!ls -lh ./

total 40
-rw-r--r--  1 kabu  staff    10K Apr  1 12:30 Run-Multiple-Intent-Experiments.ipynb
-rw-r--r--  1 kabu  staff     0B Mar 31 19:21 __init__.py
-rw-r--r--  1 kabu  staff   213B Mar 31 17:41 config1.yml
-rw-r--r--  1 kabu  staff   149B Mar 31 17:12 config2.yml


In [7]:
intent_experiment.multirun(configs=configs, out_dir='./test')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [01:22<00:00, 41.47s/it]


### Inspect the Results

In [8]:
!ls -lh ./test/intent_experiment__examples-rules-nlu

total 0
drwxr-xr-x  7 kabu  staff   224B Apr  1 12:32 config:another-very-interesting-config-because-reasons,drop:2,exclude:0.0__2022-04-01_12-31-43
drwxr-xr-x  6 kabu  staff   192B Apr  1 12:31 config:interesting-config-name1,drop:2,exclude:0.0__2022-04-01_12-31-15


In [15]:
import glob
example = list(glob.glob("./test/intent_experiment__examples-rules-nlu/*"))[1]

In [16]:
!ls -lha {example}

total 8
drwxr-xr-x  7 kabu  staff   224B Apr  1 12:32 .
drwxr-xr-x  4 kabu  staff   128B Apr  1 12:31 ..
drwxr-xr-x  5 kabu  staff   160B Apr  1 12:31 .hydra
drwxr-xr-x  4 kabu  staff   128B Apr  1 12:32 data
-rw-r--r--  1 kabu  staff   2.9K Apr  1 12:32 intent_experiment.log
drwxr-xr-x  3 kabu  staff    96B Apr  1 12:32 model
drwxr-xr-x  6 kabu  staff   192B Apr  1 12:32 report


In [17]:
!head -n 5 {example}/intent_experiment.log

[2022-04-01 12:31:43,752][leaderboard.nlu.base.base_experiment][INFO] - Running Experiment IntentExperiment
[2022-04-01 12:31:43,752][leaderboard.nlu.base.base_experiment][INFO] - Description: Experiment where we 1. exclude all intents with less than a fixed number of examples and 2. during training also remove a fixed percentage of the training examples before training.
[2022-04-01 12:31:44,610][rasa.engine.training.hooks][INFO] - Starting to train component 'LexicalSyntacticFeaturizer'.
[2022-04-01 12:31:44,661][rasa.engine.training.hooks][INFO] - Finished training component 'LexicalSyntacticFeaturizer'.
[2022-04-01 12:31:44,786][rasa.engine.training.hooks][INFO] - Starting to train component 'CountVectorsFeaturizer'.


In [18]:
!tail -n 5 {example}/intent_experiment.log

 [ 0  1  3  0  1 10  3  2  6]
 [ 0  0  0  0  0  1  1  0  2]
 [ 0  0  0  0  0  0  2  1  1]
 [ 0  0  0  0  0  0  2  1  2]]
[2022-04-01 12:32:32,713][rasa.nlu.test][INFO] - Entity evaluation results:


In [19]:
!cat {example}/.hydra/config.yaml

description: Basic Intent Experiment
model:
  name: another-very-interesting-config-because-reasons
  config_path: ./config1.yml
data:
  name: examples-rules-nlu
  domain_path: ../../../examples/rules/domain.yml
  data_path: ../../../examples/rules/data/nlu.yml
clear_rasa_cache: true
drop_intents_with_less_than: 2
exclusion_percentage: 0.0


In [20]:
!head -n 5 {example}/.hydra/hydra.yaml
!echo '...'

hydra:
  run:
    dir: ./test/intent_experiment__${data.name}/config:${model.name},drop:${drop_intents_with_less_than},exclude:${exclusion_percentage}__${now:%Y-%m-%d_%H-%M-%S}
  sweep:
    dir: multirun/${now:%Y-%m-%d}/${now:%H-%M-%S}
...


In [22]:
!ls {example}/report/

intent_confusion_matrix.png intent_histogram.png
intent_errors.json          intent_report.json
